In [9]:
import awswrangler as wr
# Abrimos un cliente de S3
import boto3
session = boto3.Session(profile_name='arquitectura',
                        region_name='us-east-1')
s3 = session.client('s3')

## ELT

## Extract y Load

### Crear tablas en la base de datos de nuestros CSVs en el data lake

- **Tabla tipo de cambio**

In [11]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio`(
    `date` date, 
    `tipo_de_cambio` double)
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-javier/tarea6/raw/tipo_de_cambio/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, 
                         database="econ", 
                         ctas_approach=False,
                         boto3_session=session)

""


- **Tabla tasa de interés**

In [ ]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_de_interes`(
    `date` date, 
    `tasa_de_interes` double)
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-javier/tarea6/raw/tasa_de_interes/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, database="econ", ctas_approach=False)

- **Tabla inflación**

In [ ]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion`(
    `date` date, 
    `inflacion` double)
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-javier/tarea6/raw/inflacion/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, database="econ", ctas_approach=False)

## Transform

In [12]:
#Tabla que junta los datos de la inflación, la tasa de interés y el tipo de cambio
query = '''
    CREATE TABLE econ.datos_mensuales AS (
        SELECT 
            a.date,
            a.inflacion,
            b.tasa_de_interes,
            c.tipo_de_cambio
        FROM econ.inflacion AS a
        LEFT JOIN econ.tasa_de_interes AS b ON a.date = b.date
        LEFT JOIN econ.tipo_de_cambio AS c ON a.date = c.date
    )
'''

wr.athena.read_sql_query(query, 
                         database="econ", 
                         ctas_approach=False,
                         boto3_session=session)

""
